In [44]:
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 58.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [45]:
import spacy
nlp = spacy.load("en_core_web_sm")

text = """
Dave watched as the forest burned up on the hill,
only a few miles from his house. The car had
been hastily packed and Marta was inside trying to round
up the last of the pets. "Where could she be?" he wondered
as he continued to wait for Marta to appear with the pets.
"""

doc = nlp(text)
token_list = [token for token in doc]

print(token_list)

[
, Dave, watched, as, the, forest, burned, up, on, the, hill, ,, 
, only, a, few, miles, from, his, house, ., The, car, had, 
, been, hastily, packed, and, Marta, was, inside, trying, to, round, 
, up, the, last, of, the, pets, ., ", Where, could, she, be, ?, ", he, wondered, 
, as, he, continued, to, wait, for, Marta, to, appear, with, the, pets, ., 
]


Удаление стоп-слов

In [46]:
filtered_tokens = [token for token in doc if not token.is_stop]
print(filtered_tokens)

[
, Dave, watched, forest, burned, hill, ,, 
, miles, house, ., car, 
, hastily, packed, Marta, inside, trying, round, 
, pets, ., ", ?, ", wondered, 
, continued, wait, Marta, appear, pets, ., 
]


Приведение к нормальной форме

В процессе нормализации все формы слова приводятся к единому представлению. Например, watched, watching и watches после нормализации превращаются в watch. Есть два основных подхода к нормализации: стемминг и лемматизация.

In [47]:
lemmas = [
    f"Token: {token}, lemma: {token.lemma_}"
    for token in filtered_tokens
]

print(lemmas)

['Token: \n, lemma: \n', 'Token: Dave, lemma: Dave', 'Token: watched, lemma: watch', 'Token: forest, lemma: forest', 'Token: burned, lemma: burn', 'Token: hill, lemma: hill', 'Token: ,, lemma: ,', 'Token: \n, lemma: \n', 'Token: miles, lemma: mile', 'Token: house, lemma: house', 'Token: ., lemma: .', 'Token: car, lemma: car', 'Token: \n, lemma: \n', 'Token: hastily, lemma: hastily', 'Token: packed, lemma: pack', 'Token: Marta, lemma: Marta', 'Token: inside, lemma: inside', 'Token: trying, lemma: try', 'Token: round, lemma: round', 'Token: \n, lemma: \n', 'Token: pets, lemma: pet', 'Token: ., lemma: .', 'Token: ", lemma: "', 'Token: ?, lemma: ?', 'Token: ", lemma: "', 'Token: wondered, lemma: wonder', 'Token: \n, lemma: \n', 'Token: continued, lemma: continue', 'Token: wait, lemma: wait', 'Token: Marta, lemma: Marta', 'Token: appear, lemma: appear', 'Token: pets, lemma: pet', 'Token: ., lemma: .', 'Token: \n, lemma: \n']


Векторизация – преобразование токена в числовой массив, который представляет его свойства. В контексте задачи вектор уникален для каждого токена. Векторные представления токенов используются для оценки сходства слов, классификации текстов и т. д. В spaCy токены векторизуются в виде плотных массивов, в которых для каждой позиции определены ненулевые значений. Это отличает используемый подход от ранних методов, в которых для тех же целей применялись разреженные массивы и большинство позиций были заполнены нулями.

In [48]:
filtered_tokens[1].vector

array([-1.0732638 , -1.589313  , -0.7485428 ,  0.8033879 ,  0.19977242,
        0.00840408,  1.5419115 ,  0.78789234, -0.10507897, -0.08379465,
        1.6370184 ,  0.99815553, -0.27276033, -0.9078418 , -1.2485981 ,
       -0.5253064 ,  0.3660615 ,  0.32205266,  0.26947686, -0.6838585 ,
       -1.3466268 ,  0.011222  , -0.24088496, -0.48466784, -0.33174878,
       -0.05325326,  1.8773433 ,  0.56494987, -0.96057415,  0.78610957,
       -0.44939822, -1.4648833 , -0.38066617,  1.0480769 , -0.8341217 ,
       -0.2217494 , -0.85443366,  0.35594553, -0.1127467 ,  1.2787786 ,
       -0.8223141 ,  0.18473059, -0.08983876,  0.6325263 , -1.1029459 ,
        0.3719486 ,  0.11167981,  1.5298815 ,  0.73126984, -0.01238485,
       -0.38741022,  0.24374121,  0.66934216, -0.51473886, -0.05107652,
       -0.6836413 ,  1.2553529 , -0.4258146 ,  0.82571185, -0.40290013,
       -1.0714419 ,  0.8215423 ,  0.10354415, -0.5627635 ,  0.34108153,
       -0.46954852, -0.644461  , -0.4248718 , -0.74732137, -0.93

Создаем собственный анализатор тональности текстов

In [49]:
import os                                                     #

def load_training_data(
    data_directory: str = "aclImdb/train",
    split: float = 0.8,
    limit: int = 0) -> tuple:
    # Загрузка данных из файлов
    reviews = []                                              #
    for label in ["pos", "neg"]:                              #
        labeled_directory = f"{data_directory}/{label}"       #
        for review in os.listdir(labeled_directory):          #
            if review.endswith(".txt"):                       #
                with open(f"{labeled_directory}/{review}") as f:
                    text = f.read()                           #
                    text = text.replace("<br />", "\n\n")     #
                    if text.strip():                          #
                        spacy_label = {                       #
                            "cats": {                         #
                                "pos": "pos" == label,        #
                                "neg": "neg" == label         #
                            }                                 #
                        }                                     #
                        reviews.append((text, spacy_label))   #

In [50]:
import os
import random                                  #

def load_training_data(
    data_directory: str = "aclImdb/train",
    split: float = 0.8,
    limit: int = 0
) -> tuple:
    # Загрузка данных из файлов
    reviews = []
    for label in ["pos", "neg"]:
        labeled_directory = f"{data_directory}/{label}"
        for review in os.listdir(labeled_directory):
            if review.endswith(".txt"):
                with open(f"{labeled_directory}/{review}") as f:
                    text = f.read()
                    text = text.replace("<br />", "\n\n")
                    if text.strip():
                        spacy_label = {
                            "cats": {
                                "pos": "pos" == label,
                                "neg": "neg" == label}
                        }
                        reviews.append((text, spacy_label))
    random.shuffle(reviews)                    #

    if limit:                                  #
        reviews = reviews[:limit]              #
    split = int(len(reviews) * split)          #
    return reviews[:split], reviews[split:]    #

In [51]:
load_training_data(
    data_directory = "aclImdb/train",
    split = 0.8,
    limit = 0)[0][0]

('For some reason, TV Guide gave this two and a half stars, plus Faye Dunaway is in it, so it definitely looked like something to see. My, oh, my, this may be the worst film I\'ve ever seen. Ever. From its horrid acting (every time the girl asks the boy what\'s wrong with him, I shouted to the TV "I can\'t act!" When she asks what he needs, I yell "I need acting lessons!" to the unbelievably bad dialog ("Give me back my organs!").\n\n\n\nAnd the Brian DePalma wannabe ending, too, it was all just beyond awful. I wanted to like it. Dunaway is one of the best actors ever. And the production values were pretty good.\n\n\n\nBut wowzers, this had me laughing, LAUGHING!, most of the time.\n\n\n\nDon\'t even bother out of curiosity, that was my first mistake. Staying with it was definitely my second, and third and fourth.',
 {'cats': {'pos': False, 'neg': True}})

Обучение классификатора

Конвейер spaCy позволяет создать и обучить сверточную нейронную сеть (CNN) для классификации текстовых данных. Здесь этот подход используется для сентимент-анализа, но ничто не мешает распространить его и на другие задачи классификации текстов.

В этой части проекта мы выполним три шага:

    Измененим базовый конвейер spaCy для включения компонента textcat.
    Создадим цикл для обучения компонента textcat.
    Научимся оценивать прогресс обучения модели после заданного количества циклов обучения.


In [52]:
def evaluate_model(tokenizer, textcat, test_data: list) -> dict:
    reviews, labels = zip(*test_data)
    reviews = (tokenizer(review) for review in reviews)
    # Указываем TP как малое число, чтобы в знаменателе
    # не оказался 0
    TP, FP, TN, FN = 1e-8, 0, 0, 0
    for i, review in enumerate(textcat.pipe(reviews)):
        true_label = labels[i]['cats']
        score_pos = review.cats['pos']
        if true_label['pos']:
            if score_pos >= 0.5:
                TP += 1
            else:
                FN += 1
        else:
            if score_pos >= 0.5:
                FP += 1
            else:
                TN += 1
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f_score = 2 * precision * recall / (precision + recall)
    return {"precision": precision, "recall": recall, "f-score": f_score}

In [53]:
import os
import random
import spacy
from spacy.util import minibatch, compounding

def train_model(
    training_data: list,
    test_data: list,
    iterations: int = 20) -> None:
    # Строим конвейер
    nlp = spacy.load("en_core_web_sm")
    if "textcat" not in nlp.pipe_names:
        textcat = nlp.create_pipe(
            "textcat", config={"architecture": "simple_cnn"}
        )
        nlp.add_pipe(textcat, last=True)
    else:
        textcat = nlp.get_pipe("textcat")

    textcat.add_label("pos")
    textcat.add_label("neg")

    # Обучаем только textcat
    training_excluded_pipes = [
        pipe for pipe in nlp.pipe_names if pipe != "textcat"
    ]
    with nlp.disable_pipes(training_excluded_pipes):
        optimizer = nlp.begin_training()
        # Training loop
        print("Начинаем обучение")
        print("Loss\t\tPrec.\tRec.\tF-score")          #
        batch_sizes = compounding(
            4.0, 32.0, 1.001
        )  # Генератор бесконечной последовательности входных чисел
        for i in range(iterations):
            loss = {}
            random.shuffle(training_data)
            batches = minibatch(training_data, size=batch_sizes)
            for batch in batches:
                text, labels = zip(*batch)
                nlp.update(
                    text,
                    labels,
                    drop=0.2,
                    sgd=optimizer,
                    losses=loss
                )
            with textcat.model.use_params(optimizer.averages):
                evaluation_results = evaluate_model(   #
                    tokenizer=nlp.tokenizer,           #
                    textcat=textcat,                   #
                    test_data=test_data                #
                )                                      #
                print(f"{loss['textcat']:9.6f}\t\
{evaluation_results['precision']:.3f}\t\
{evaluation_results['recall']:.3f}\t\
{evaluation_results['f-score']:.3f}")

    # Сохраняем модель                                 #
    with nlp.use_params(optimizer.averages):           #
        nlp.to_disk("model_artifacts")                 #

In [54]:
train, test = load_training_data(limit=20000)
train_model(train, test, iterations=1)

ConfigValidationError: 

Config validation error
textcat -> architecture	extra fields not permitted
{'nlp': <spacy.lang.en.English object at 0x7f29ee3bbb50>, 'name': 'textcat', 'architecture': 'simple_cnn', 'model': {'@architectures': 'spacy.TextCatEnsemble.v2', 'linear_model': {'@architectures': 'spacy.TextCatBOW.v3', 'exclusive_classes': True, 'length': 262144, 'ngram_size': 1, 'no_output_layer': False}, 'tok2vec': {'@architectures': 'spacy.Tok2Vec.v2', 'embed': {'@architectures': 'spacy.MultiHashEmbed.v2', 'width': 64, 'rows': [2000, 2000, 500, 1000, 500], 'attrs': ['NORM', 'LOWER', 'PREFIX', 'SUFFIX', 'SHAPE'], 'include_static_vectors': False}, 'encode': {'@architectures': 'spacy.MaxoutWindowEncoder.v2', 'width': 64, 'window_size': 1, 'maxout_pieces': 3, 'depth': 2}}}, 'scorer': {'@scorers': 'spacy.textcat_scorer.v2'}, 'threshold': 0.0, '@factories': 'textcat'}

In [55]:
TEST_REVIEW = """
Transcendently beautiful in moments outside the office, it seems almost
sitcom-like in those scenes. When Toni Colette walks out and ponders
life silently, it's gorgeous.<br /><br />The movie doesn't seem to decide
whether it's slapstick, farce, magical realism, or drama, but the best of it
doesn't matter. (The worst is sort of tedious - like Office Space with less humor.)
"""

In [56]:
def test_model(input_data: str):
    # Загружаем сохраненную модель
    loaded_model = spacy.load("model_artifacts")
    parsed_text = loaded_model(input_data)
    # Определяем возвращаемое предсказание
    if parsed_text.cats["pos"] > parsed_text.cats["neg"]:
        prediction = "Положительный отзыв"
        score = parsed_text.cats["pos"]
    else:
        prediction = "Негативный отзыв"
        score = parsed_text.cats["neg"]
    print(f"Текст обзора: {input_data}\n\
Предсказание: {prediction}\n\
Score: {score:.3f}")

In [59]:
test_model(input_data=TEST_REVIEW)

OSError: [E050] Can't find model 'model_artifacts'. It doesn't seem to be a Python package or a valid path to a data directory.

In [60]:
import pandas as pd
import sklearn.metrics

df = pd.read_csv("kinopoisk.zip", index_col=0)
df.sample(frac=1)[:5]

,review,translation,type
2648,"Это было.\n\n\nОдин из немногих случаев, когда...",It was.\n\n\nOne of the few times when I'm con...,neutral
489,Своё знакомство с сериалом я начал лишь по око...,I started my acquaintance with the series only...,good
1897,Фильм «Игры разума» или «Прекрасный разум» (в ...,"The film ""A Beautiful Mind"" or ""Beautiful Mind...",bad
70,Фильм 'Красавица и чудовище' - визуальный шеде...,Beauty and the Beast is a visual masterpiece. ...,good
460,"Что удивительно, но с этим сериалом я познаком...","Surprisingly, I only got acquainted with this ...",good


In [61]:
def test_model(input_data):
    loaded_model = spacy.load("model_artifacts")
    parsed_text = loaded_model(input_data)
    if parsed_text.cats["pos"] > parsed_text.cats["neg"]:
        prediction = "good"
        score = parsed_text.cats["pos"]
    else:
        prediction = "bad"
        score = parsed_text.cats["neg"]
    return prediction

In [62]:
df['pred'] = pred
yy = df.iloc[:1999]
y_true = yy['type']
y_pred = yy['pred']

y_true[y_true == 'good'] = 1
y_true[y_true == 'bad'] = 0
y_pred[y_pred == 'good'] = 1
y_pred[y_pred == 'bad'] = 0

accuracy = sklearn.metrics.accuracy_score(y_true.astype(int), y_pred.astype(int))
f_score = sklearn.metrics.f1_score(y_true.astype(int), y_pred.astype(int))

print(f"Метрика accuracy составила: {accuracy:.3f}.")
print(f"F1-мера модели равна {f_score:.3f}.")

NameError: name 'pred' is not defined

In [ ]:
df[df['type'] == 'neutral']['pred'].value_counts()